# Creds

> Fill in a module description here


In [16]:
# | default_exp google.auth

In [17]:
# | exporti

import os
import json
from dataclasses import dataclass, field
from typing import List

from dotenv import load_dotenv, set_key

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

from nbdev.showdoc import patch_to

In [18]:
# | hide
from nbdev.showdoc import show_doc

In [19]:
# | export
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import Resource

In [20]:
# | export

DEFAULT_SCOPE = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive.metadata.readonly",
    "https://www.googleapis.com/auth/drive.readonly",
]

In [21]:
# | exports
class GoogleAuth_NoEnv_Error(Exception):
    def __init__(self, env_token):
        super().__init__(f"{env_token} not found in ENV")

In [22]:
# | export


@dataclass
class GoogleAuth:
    creds: Credentials = None
    service: Resource = None

    scope: List[str] = field(default_factory=lambda: DEFAULT_SCOPE)

    def _refresh_token(self) -> Credentials:
        """Refreshes the token or creates a new one if it does not exist"""

        if (
            self.creds
            and isinstance(self.creds, Credentials)
            and self.creds.expired
            and self.creds.refresh_token
        ):
            # if creds token can be refreshed, do it.
            try:
                print("refreshing creds using saved token")
                self.creds.refresh(Request())

            except RefreshError as e:
                print(e)
                self.creds.refresh(Request())

        self.generate_service(service_name="drive", service_version="v3")

        return self.creds

    def _new_token_via_auth_flow(self, flow):
        # If there are no (valid) credentials available, run the Oauth flow
        print("generating a brand new token cred via auth flow")

        self.creds = flow.run_local_server(port=0)

        self._refresh_token()

        return self.creds

    def generate_service(self, service_name="drive", service_version="v3") -> Resource:
        """Generates a service object for the given service name and version"""

        print(f"generating service object on {self.__class__.__name__}")

        self.service = build(
            serviceName=service_name, version=service_version, credentials=self.creds
        )

        return self.service

# Get Cred from ENV

In [23]:
# |exporti


@patch_to(GoogleAuth, cls_method=True)
def get_creds_from_env(
    cls: GoogleAuth,
    scope=None,
    credentials_env_key: str = "GDOC_CLIENT",
    token_env_key: str = None,
):
    """
    credentials should be stored in the .env file as a json (NOT A STRING)
    token will be stored as a string (NOT A JSON OBJ)

    generate creds here, https://console.cloud.google.com/apis/credentials/oauthclient
    for a new implementation you'll need to
    1. set up a project at console.cloud.google.com,
    2. enable (turn on) the appropriate APIs for that project
    3. configure OAuth access (your python script will use an Oauth token to authenticate, so your project needs the OAuth screen)
    4. download the creds file and store it as env_creds.json
    """

    scope = scope or DEFAULT_SCOPE

    credentials_str = os.getenv(credentials_env_key)

    if not credentials_str:
        raise GoogleAuth_NoEnv_Error(credentials_env_key)

    credentials_json = (
        json.loads(credentials_str)
        if credentials_str and isinstance(credentials_str, str)
        else credentials_str
    )

    if token_env_key:
        token_str = os.getenv(token_env_key)

        if not token_str:
            raise GoogleAuth_NoEnv_Error(token_env_key)

        token_json = (
            json.loads(token_str)
            if token_str and isinstance(token_str, str)
            else token_str
        )

        print("using saved token")
        ga_auth = cls(
            creds=Credentials.from_authorized_user_info(token_json, scope), scope=scope
        )
        ga_auth._refresh_token()
        return ga_auth

    flow = InstalledAppFlow.from_client_config(credentials_json, scope)

    ga_auth = cls(scope=scope)
    ga_auth._new_token_via_auth_flow(flow)

    return ga_auth

#### sample implementation of get_creds_from_env


In [24]:
from dotenv import load_dotenv

load_dotenv(".env")

GoogleAuth.get_creds_from_env(
    credentials_env_key="GDOC_KEY",
    token_env_key="GDOC_TOKEN",
)

using saved token
generating service object on GoogleAuth


GoogleAuth(creds=<google.oauth2.credentials.Credentials object at 0x7f3df4d7f280>, service=<googleapiclient.discovery.Resource object at 0x7f3df4560730>, scope=['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/drive.metadata.readonly', 'https://www.googleapis.com/auth/drive.readonly'])

In [25]:
import datetime as dt
from dotenv import set_key, load_dotenv

In [26]:
# | hide
# setup get_creds_from_file by exporting working creds to the env

from gdoc_sync.utils import update_env
import json

from dotenv import load_dotenv

load_dotenv(".env")

# create credentials file
credentials_env_key = "GDOC_KEY"
credentials_env = os.environ.get(credentials_env_key)
credentials_file_path = "env_creds.json"

with open(credentials_file_path, "w") as f:
    json.dump(json.loads(credentials_env), f)


# create token file from valid GoogleAuth object
token_key = "GDOC_TOKEN"
token_file_path = "env_token.json"

ga_auth = GoogleAuth.get_creds_from_env(
    credentials_env_key=credentials_env_key,
    token_env_key=token_key,
)

update_env(".env", token_key, json.loads(ga_auth.creds.to_json()), debug_prn=False)

token_env = os.environ.get(token_key)

with open(token_file_path, "w") as f:
    json.dump(json.loads(token_env), f)

using saved token
generating service object on GoogleAuth


# Get Creds from File

In [27]:
# |exporti


@patch_to(GoogleAuth, cls_method=True)
def get_creds_from_file(
    cls: GoogleAuth,
    credentials_file_path="env_creds.json",
    token_file_path=None,
    scope=None,
) -> Credentials:
    """
    generate creds here, https://console.cloud.google.com/apis/credentials/oauthclient
    for a new implementation you'll need to
    1. set up a project at console.cloud.google.com,
    2. enable (turn on) the appropriate APIs for that project
    3. configure OAuth access (your python script will use an Oauth token to authenticate, so your project needs the OAuth screen)
    4. download the creds file and store it as env_creds.json
    """

    scope = scope or DEFAULT_SCOPE

    if token_file_path and os.path.exists(token_file_path):
        print(f"using saved token {token_file_path}")
        ga_auth = cls(
            creds=Credentials.from_authorized_user_file(token_file_path, scope),
            scope=scope,
        )
        ga_auth._refresh_token()
        return ga_auth

    flow = InstalledAppFlow.from_client_secrets_file(credentials_file_path, scope)

    ga_auth = cls(scope=scope)
    ga_auth._new_token_via_auth_flow(flow)
    return ga_auth

In [28]:
show_doc(GoogleAuth.get_creds_from_file)

---

[source](https://github.com/jaewilson07/gdoc_sync/blob/main/gdoc_sync/google/auth.py#L133){target="_blank" style="float:right; font-size:smaller"}

### GoogleAuth.get_creds_from_file

>      GoogleAuth.get_creds_from_file (credentials_file_path='env_creds.json',
>                                      token_file_path=None, scope=None)

generate creds here, https://console.cloud.google.com/apis/credentials/oauthclient
for a new implementation you'll need to
1. set up a project at console.cloud.google.com,
2. enable (turn on) the appropriate APIs for that project
3. configure OAuth access (your python script will use an Oauth token to authenticate, so your project needs the OAuth screen)
4. download the creds file and store it as env_creds.json

#### sample implementation of get_creds_from_file

In [29]:
from dotenv import load_dotenv

load_dotenv(".env")

GoogleAuth.get_creds_from_file(
    credentials_file_path="env_creds.json", token_file_path="env_token.json", scope=None
)

using saved token env_token.json
generating service object on GoogleAuth


GoogleAuth(creds=<google.oauth2.credentials.Credentials object at 0x7f3df45606a0>, service=<googleapiclient.discovery.Resource object at 0x7f3df4561180>, scope=['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/drive.metadata.readonly', 'https://www.googleapis.com/auth/drive.readonly'])

In [30]:
# | hide
import nbdev

nbdev.nbdev_export()